In [1]:
import azureml.core

source_folder = '.'
print("SDK version:", azureml.core.VERSION)

SDK version: 1.0.15


## Load Workspace
The following cell will read the workspace configuration from aml_config/config.json

Make sure to place the config.json file containing your workspace configuration there -- see <a href="https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.workspace(class)?view=azure-ml-py#from-config-path-none--auth-none-">here</a> for details. 

In [4]:
from azureml.core import Workspace
ws = Workspace.from_config()

Found the config file in: /Users/danielsc/git/azureml-and-r/aml_config/config.json


## Create and Experiment 

In [5]:
from azureml.core import Experiment

experiment = Experiment(ws, 'testing-R')

## The R file
There is an R file in this folder which just loads some data and creates a plot. The next cell will show the file, so you can inspect it. The script writes a pdf to the `outputs` folder -- that file is uploaded to the run history after the run completes.
In order to log data to AML you need the azureml Run object. Since that lives in python, we are using reticulate to spin up a python environment where we can create/fetch it. 



In [6]:
!pygmentize test.R

library(caret)
library(reticulate)

azureml_core = import("azureml.core")
run = azureml_core$Run$get_context()
run$log("test", 1)

print(c("runid",run$id))

# attach the iris dataset to the environment
data(iris)
# rename the dataset
dataset <- iris

# list types for each attribute
sapply(dataset, class)

# take a peek at the first 5 rows of the data
head(dataset)

# list the levels for the class
levels(dataset$Species)

# summarize the class distribution
percentage <- prop.table(table(dataset$Species)) * 100
cbind(freq=table(dataset$Species), percentage=percentage)

# summarize attribute distributions
summary(dataset)

# split input and output
x <- dataset[,1:4]
y <- dataset[,5]

# density plots for each attribute by class value
scales <- list(x=list(relation="free"), y=list(relation="free"))

pdf('outputs/plots.pdf')
featurePlot(x=x, y=y, plot="density", scales=scales)

run$'_client'$flush()


In [12]:
target = ws.compute_targets['aml-compute']

In [13]:
from azureml.train.estimator import Estimator

estwithr = Estimator(source_directory='.', 
                     conda_packages=['r-essentials', 'mro-base', 'r-reticulate'],
                    compute_target=target, 
                    entry_script='launchR.py', 
                     script_params={'test.R': ''})

runwithr = experiment.submit(estwithr)
runwithr

Experiment,Id,Type,Status,Details Page,Docs Page
testing-R,testing-R_1551475722_b6601c5f,azureml.scriptrun,Queued,Link to Azure Portal,Link to Documentation


In [14]:
from azureml.widgets import RunDetails
RunDetails(runwithr).show()

_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', '…